In [1]:
import folium

In [2]:
m = folium.Map(location=[28.116667, -17.216667])
m
#m.save("map_folium.html")

In [3]:
#m = folium.Map(location=[28.116667, -17.216667], tiles="https://server.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}", zoom_start=10,attr="Tiles &copy; Esri &mdash; Sources: GEBCO, NOAA, CHS, OSU, UNH, CSUMB, National Geographic, DeLorme, NAVTEQ, and Esri")
#m = folium.Map(location=[28.116667, -17.216667], tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}", zoom_start=13,attr="Tiles &copy; Esri &mdash; Sources: GEBCO, NOAA, CHS, OSU, UNH, CSUMB, National Geographic, DeLorme, NAVTEQ, and Esri")
m = folium.Map(location=[28.116667, -17.216667], tiles="https://tile.opentopomap.org/{z}/{x}/{y}.png", zoom_start=11,attr="OpenTopoMap (CC-BY-SA)")

m.add_child(folium.LatLngPopup())

m

In [4]:
m = folium.Map(location=[28.116667, -17.216667], tiles="https://tiles.emodnet-bathymetry.eu/2020/baselayer/web_mercator/{z}/{x}/{y}.png", zoom_start=11,attr="Emodnet bathymetry")
m

In [5]:
map_geo = folium.Map(location=[28.116667, -17.216667])
folium.raster_layers.WmsTileLayer(url = 'https://ows.emodnet-bathymetry.eu/wms?VERSION%3D1.3.0',
                                  layers = 'emodnet:contours',
                                  transparent = True, 
                                  control = True,
                                  fmt="image/png",
                                  name = 'bathymetry',
                                  overlay = True,
                                  show = True,
                                  ).add_to(map_geo)
folium.LayerControl().add_to(map_geo)
map_geo

In [6]:
import pandas as pd
import folium
from folium.plugins import BeautifyIcon

variables = ['Long', 'Lat', 'Year', 'Name', 'Shore distance','Depth','Date','WKT']

df_gm = pd.read_csv('data/gm.csv', sep=';')
df_gm.columns = variables
df_sf = pd.read_csv('data/sf.csv', sep=';')
df_sf.columns = variables
df_tt = pd.read_csv('data/tt.csv', sep=';')
df_tt.columns = variables

frames = [df_gm, df_sf, df_tt]

df = pd.concat(frames)

# Renaming columns
df.columns = ['Long', 'Lat', 'Year', 'Name', 'Shore distance','Depth','Date','WKT']

map = folium.Map(location=[df.Lat.mean(), df.Long.mean()], 
                 zoom_start=8, 
                 control_scale=True)
#Add a single marker

folium.Marker(location=[df.Lat.mean(), df.Long.mean()], icon=folium.Icon(color='green', icon='record')).add_to(map_geo)

#Loop through each row in the dataframe
for i,row in df.iterrows():
    #Setup the content of the popup
    iframe = folium.IFrame('Depth:' + str(row["Depth"]))
    
    #Initialise the popup using the iframe
    popup = folium.Popup(iframe, min_width=300, max_width=300)
    
    #Add each row to the map
    folium.Marker(location=[row['Lat'],row['Long']],
                  popup = popup, c='Name',icon=BeautifyIcon(
    icon_shape='rectangle-dot', 
    border_color='yellow', 
    border_width=3,
)).add_to(map_geo)
map_geo

In [7]:
from scipy.stats import chi2_contingency

df["DepthQ"]=pd.qcut(df.Depth,4,labels=["D1","D2",
   "D3","D4"])

df['DepthC'] = pd.cut(x=df['Depth'], bins=[-100,250, 500,
                                          1000, 2000,4000],labels=[ '250','500','1000','2000','4000'])
#df["DepthQ"].describe()
#df
#print(df['DepthC'].unique())
obs = pd.crosstab(df['Name'], df['DepthC'])
print(obs)

chi2, p, dof, ex = chi2_contingency(obs, correction=False)
print (f"chi2 = {chi2}, p= {p}, dof= {dof}")
print (ex)

DepthC                      250  500  1000  2000  4000
Name                                                  
Globicephala macrorhynchus   10    1    11    83     5
Stenella frontalis           55   16    46    48    26
Tursiops truncatus           42   23    49    30     4
chi2 = 121.8246349647116, p= 1.392002430524872e-22, dof= 8
[[26.21380846  9.79955457 25.9688196  39.44320713  8.57461024]
 [45.51670379 17.0155902  45.09131403 68.48775056 14.88864143]
 [35.26948775 13.18485523 34.93986637 53.06904232 11.53674833]]
